In [1]:
import catboost as cb
from sklearn.metrics import mean_squared_error
import optuna
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

C:\Users\79213\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data = pd.read_excel('prelast.xlsx',skiprows=2)
dat = data.drop(columns=['неделя'])
dat['Начало нед'] = pd.to_datetime(dat['Начало нед'].values)
dat = data.fillna(0)
val = dat.iloc[244:]
dat = dat.iloc[:244]
train, test = train_test_split(dat,
                test_size=0.2,
                train_size=0.8,
                shuffle=True)
train = train.drop(columns=['Начало нед'])
test = test.drop(columns=['Начало нед'])
tr = Pool(train.drop(columns='Продажи, рубли'),label=train['Продажи, рубли'].values)
ts = Pool(test.drop(columns='Продажи, рубли'),label=test['Продажи, рубли'].values)

In [ ]:
def objective(trial):
    params = {
        "iterations": 10_000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.1, log=True),
        "depth": trial.suggest_int("depth", 6, 14),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 500),
        "grow_policy": trial.suggest_categorical("grow_policy", ["Lossguide", "Depthwise", "SymmetricTree"]),
        "loss_function": trial.suggest_categorical('loss_function',['MAPE','RMSE','MAE']),
        "l2_leaf_reg": trial.suggest_int('l2_leaf_reg',2,10)
    }

    model = cb.CatBoostRegressor(**params, silent=True)
    model = model.fit(tr)
    predictions = model.predict(ts)
    rmse = mean_squared_error(test['Продажи, рубли'].values, predictions, squared=False)
    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=200)